<a href="https://colab.research.google.com/github/Babaevskiyy/SystemMonitor/blob/main/Faceswap_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

Faceswap - Google Colab

You also need Faceswap GUI for this to work, since you can't extract faces here, yet.

Files you need to upload on Google Drive
1.   face_a.zip (source face pics you want to swap)
2.   face_b.zip (faces you want to get swapped)

> Zipped files with extracted faces and alignments.fsa

3.   train.ini
> Your train.ini from your local system, in Windows this is in: C:\Users\YourName\faceswap\config

Here is what filesystem looks like in Google Drive
*   /colab_files/faceswap/faces/face_a.zip
*   /colab_files/faceswap/faces/face_b.zip
*   /colab_files/faceswap/config/train.ini

Recommended folders to create on Google Drive:
*   /colab_files/faceswap/models/YourModelName
*   /colab_files/faceswap/output/timelapse

Here is what filesystem should look like in this Google Colab after the "Mount Google Drive" step below.
*   /content/drive/My Drive/colab_files/faceswap/faces/face_a.zip
*   /content/drive/My Drive/colab_files/faceswap/faces/face_b.zip
*   /content/drive/My Drive/colab_files/faceswap/config/train.ini

# Usage Notes

This is my personal SOP for handling the Faceswap process. I find it is best to tweak your settings, extract, test your training, and convert everything using your local system.

But training on my laptop took weeks to get the same results that you can get from just days by using a Google Colab.

*   Extract, Align, then Zip all Faces A
*   Extract, Align, then Zip all Faces B
*   Modify the "Run Training" script below to your preferences
*   Train in this Colab Notebook
*   Watch Google Drive timestamp folder to see how the training is improving
*   Download YourModelName to your local system
*   Convert using the downloaded model


# Keep Session Alive
 (it may not work due to latest Google Colab Captcha changes.)



In [1]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# Check your GPU
If it isn't **Tesla T4** or **Tesla P100**, go Runtime->Factory Reset Runtime until you get one.

In [2]:
!nvidia-smi

import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

Sun Aug 17 12:20:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Setup Faceswap

In [3]:
#@title Set Time Zone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Turkey /etc/localtime
!date

#above is for HST, you can find yours in
#/usr/share/zoneinfo


Sun Aug 17 03:20:36 PM +03 2025


In [3]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
#@title Get your training data(aka the faces you extracted in app)
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip" .

!unzip face_a.zip -d face_a
!unzip face_b.zip -d face_b

!rm face_a.zip
!rm face_b.zip


Archive:  face_a.zip
  inflating: face_a/025_0.png        
  inflating: face_a/006_0.png        
  inflating: face_a/007_0.png        
  inflating: face_a/008_0.png        
  inflating: face_a/009_1.png        
  inflating: face_a/010_1.png        
  inflating: face_a/011_0.png        
  inflating: face_a/012_0.png        
  inflating: face_a/013_0.png        
  inflating: face_a/014_0.png        
  inflating: face_a/015_0.png        
  inflating: face_a/016_0.png        
  inflating: face_a/017_0.png        
  inflating: face_a/018_0.png        
  inflating: face_a/019_0.png        
  inflating: face_a/020_0.png        
  inflating: face_a/021_0.png        
  inflating: face_a/022_0.png        
  inflating: face_a/023_0.png        
  inflating: face_a/024_0.png        
Archive:  face_b.zip
  inflating: face_b/SnapReels.Net_AQOyVn0bFsuKONa7x2gnBWeRG4nPpeMeSbGtQkvMPX6LxRHvbLc-fVgir9f0YluxqRk7JZ4kyq9_R1l_q-x5RIb9ftq6jQsGO62smoo_000403_0.png  
  inflating: face_b/SnapReels.Net_AQOyVn0bFsu

In [5]:
#@title Get the latest Faceswap
!git clone https://github.com/deepfakes/faceswap.git

Cloning into 'faceswap'...
remote: Enumerating objects: 14700, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 14700 (delta 25), reused 20 (delta 20), pack-reused 14661 (from 3)
Receiving objects: 100% (14700/14700), 198.72 MiB | 34.51 MiB/s, done.
Resolving deltas: 100% (10489/10489), done.


In [6]:
#@title Copy your configuration(train.ini) file
!cp "/content/drive/My Drive/colab_files/faceswap/config/train.ini" faceswap/config/
!ls -lA faceswap/config/
!cat faceswap/config/train.ini

total 108
-rw-r--r-- 1 root root      0 Aug 17 12:50 .keep
-rw------- 1 root root 109763 Aug 17 12:50 train.ini
[global]
# НАСТРОЙКИ, ПРИМЕНИМЫЕ КО ВСЕМ МОДЕЛЯМ
# ПРИМЕЧАНИЕ: ДО ТЕХ ПОР, ПОКА ОБ ЭТОМ НЕ СКАЗАНО, ЗНАЧЕНИЯ, ИЗМЕНЕННЫЕ ЗДЕСЬ, БУДУТ ПРИМЕНЕНЫ ПРИ
# СОЗДАНИИ НОВОЙ МОДЕЛИ.

# Как централизовывать тренировочное изображение. Центр в извлеченных изображениях находится в
# середине черепа, основанный на примерной позе лица. Подсекция этих изображений используется для
# тренировки. Используемый центр диктует то, как эта подсекция будет обрезана из выравненных
# изображений.
#     - face: Центрирует учебное изображение по центру лица, регулируя угол наклона и поворота.
#     - head: Централизует тренировочное изображение в центре головы, регулируя угол наклона и
# 		поворота. Примечание: Следует выбирать централизацию головы, если вы планируете включать голову
# 		полностью (включая волосы) в финальную замену. Может дать смешанные результаты. В дополнении, оно
# 		стоит того тольк

In [8]:
#@title Install Tensorflow

!pip install -r /content/faceswap/requirements/requirements_nvidia.txt

Ignoring pywin32: markers 'sys_platform == "win32"' don't match your environment
Ignoring pynvx: markers 'sys_platform == "darwin"' don't match your environment
INFO: pip is looking at multiple versions of fastcluster to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 142.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 49.8 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 73.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 132.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 64.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 116.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 47.0 MB/s  0:00:00
  Attempting u

In [9]:
%cd ../..

/content


In [10]:
!python3 faceswap/tools.py mask -a 'face_b/alignments.fsa' -i 'face_b' -it faces -M vgg-clear -p all -o '/drive/MyDrive/colab_files/faceswap/faces/facexb/mask' -b 3 -t 4 -ot mask -L INFO


First time configuration. Please select the required backend
1: CPU, 2: DIRECTML, 3: NVIDIA, 4: APPLE SILICON, 5: ROCM: 3
Faceswap config written to: /content/faceswap/config/.faceswap
Setting Faceswap backend to NVIDIA
usage: tools.py
       [-h]
       {alignments,effmpeg,manual,mask,model,preview,sort}
       ...

positional arguments:
  {alignments,effmpeg,manual,mask,model,preview,sort}
    alignments
    This
    command
    lets you
    perform
    various
    tasks
    pertaining
    to an
    alignments
    file.
    effmpeg
    This
    command
    allows you
    to easily
    execute
    common
    ffmpeg
    tasks.
    manual
    This
    command
    lets you
    perform
    various
    actions on
    frames,
    faces and
    alignments
    files using
    visual
    tools.
    mask
    This tool
    allows you
    to
    generate,
    import,
    export or
    preview
    masks for
    existing
    alignments.
    model
    This tool
    lets you
    perform
    actions o

# Run Training
Iteration number is up to you but it's recommended that you should do until the number you are okay with. You should check your timelapse folder for that.

And if my calculations are right, trainer is doing nearly 360 iterations in 10 mins, which means 2160 iterations in 1 hr.

In [13]:
#set variables start
num_iterations = "100000"
save_every = "360"
save_model_every = "25000"
batch_num = "16"
num_gpus = "1"

trainer_type = "villain"

model_dir = "/content/drive/My Drive/colab_files/faceswap/models/YourModelName"
alignments_file_a = "face_a/2_alignments.fsa"
alignments_file_b = "face_b/alignments.fsa"
timelapse_dir = "/content/drive/My Drive/colab_files/faceswap/output/timelapse"
#set variables end

# фикс matplotlib backend (Colab без GUI)
import matplotlib
matplotlib.use("Agg")

# запуск тренировки
!python3 faceswap/faceswap.py train \
  -A 'face_a' \
  -B 'face_b'  \
  -m "{model_dir}" \
  -t "{trainer_type}" \
  -bs "{batch_num}" \
  -it "{num_iterations}" \
  -s "{save_every}" \
  -ss "{save_model_every}" \
  -tia 'face_a' \
  -tib 'face_b' \
  -to "{timelapse_dir}"


Setting Faceswap backend to NVIDIA
08/17/2025 12:52:18 INFO     Log level set to: INFO
08/17/2025 12:52:20 ERROR    Got Exception on main handler:
Traceback (most recent call last):
  File "/content/faceswap/lib/cli/launcher.py", line 223, in execute_script
    script = self._import_script()
  File "/content/faceswap/lib/cli/launcher.py", line 53, in _import_script
    module = import_module(mod)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/facesw

In [12]:
import matplotlib
matplotlib.use("Agg")


In [19]:
!pip uninstall -y tensorflow tensorflow-gpu keras keras-preprocessing

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0


In [8]:
!pip install tensorflow==2.10.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 19.5 MB/s  0:00:16
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 159.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 154.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 147.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 134.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 209.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 196.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 47.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 142.5 MB/s  0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1

In [7]:
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

--2025-08-17 12:42:52--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2148718 (2.0M) [text/x-python]
Saving to: ‘get-pip.py.1’

get-pip.py.1        100%[===================>]   2.05M  --.-KB/s    in 0.04s   

2025-08-17 12:42:53 (49.7 MB/s) - ‘get-pip.py.1’ saved [2148718/2148718]

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [2]:
!python --version

Python 3.10.12


In [1]:
# --- Переключаемся на Python 3.10 + ставим pip + TensorFlow 2.10 ---
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-dev python3.10-distutils

# Ставим Python 3.10 как основной
!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.10 2
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2
!sudo update-alternatives --set python /usr/bin/python3.10
!sudo update-alternatives --set python3 /usr/bin/python3.10

# Ставим свежий pip под 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# Проверка версии Python и pip
!python --version
!pip --version

# Ставим TensorFlow 2.10.0 (GPU версия)
!pip install tensorflow==2.10.0




Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,937 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,271 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

# Convert

In [ ]:
!python3 faceswap/faceswap.py convert -i '/content/drive/My Drive/colab_files/faceswap/Editor.avi' -o '/content/drive/My Drive/colab_files/faceswap/Results' -al '/content/drive/My Drive/colab_files/faceswap/2_alignments.fsa' -m '/content/drive/My Drive/colab_files/faceswap/models/YourModelName' -c match-hist -M none -w ffmpeg -osc 100 -l 0.4 -j 0 -t realface -L INFO